# Text to XML

## Import

### Libraries

In [1]:
import os 
import codecs
from lxml import etree

from backend import *

### Definitions

In [2]:
texts = '../texts/fiction/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Primary Texts

In [7]:
motherTextDf = textRegularize(libTextsDf, 6)
motherTextDf

/home/xtra/code/diss/codebase/backend.py:172: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


,chapID,para,text,paraID
0,1,1,"Каждый день над рабочей слободкой, в дымном, м...",1
1,1,2,"Вечером, когда садилось солнце, и на стеклах д...",2
2,1,3,"День проглочен фабрикой, машины высосали из му...",3
3,1,4,"По праздникам спали часов до десяти, потом люд...",4
4,1,5,"Усталость, накопленная годами, лишала людей ап...",5
...,...,...,...,...
4490,58,90,Ударили по руке.,4432
4491,58,91,"-- Только злобы накопите, безумные! На вас она...",4433
4492,58,92,Жандарм схватил ее за горло и стал душить. Она...,4434
4493,58,93,-- Несчастные...,4435


### Make XML

In [ ]:
root = etree.Element("text")
print(root.tag)
pt = ch = cn = pa = pn = 0
nameDict = motherDf.chap.to_dict()
for part in motherXmlDf.part.unique():
    print(f"Part {part}")
    root.append(etree.Element("part", n=str(part), name=str(part)))
    chapList = motherXmlDf.loc[motherXmlDf['part'] == part].chapID.unique()
    #print(chapList)
    for chap in chapList:
        #print(f"Chap {chap}")
        root[pt].append(etree.Element("chapter", n=str(cn+1), name=nameDict.get(chap)))
        paraList = motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chapID'] == chap)].index
        #print(paraList)
        for paragraph in paraList:
            #print(f"Paragraph {paragraph}")
            root[pt][ch].append(etree.Element("paragraph", n=str(pn+1), name=str(pa+1)))
            paraText = motherXmlDf.loc[paragraph].text
            #print(f"paraText: {paraText}")
            #print(f"pt = {pt}; ch = {ch}; paragraph = {paragraph}")
            root[pt][ch][pa].text = paraText
            pa+=1
            pn+=1
        pa=0
        ch+=1
        cn+=1
    ch=0
    pt+=1
#t+=1
#print(etree.tostring(root, pretty_print=True, xml_declaration=True))
#etree.ElementTree(root).write('../site/texts/mother.xml', pretty_print=True, xml_declaration=True, encoding='windows-1251')

In [47]:
root = etree.Element("prose")
print(root.tag)
pt = ch = pa = 1
#TestList = [x for x in bibliiaDf.test.unique()]
#for test in range(len(TestList)): 
#    root.append(etree.Element("t", n=str(t), name=TestList[test]))

for part in motherXmlDf.part.unique(): 
    print(f"Part {part}")
    ChapList = list(motherXmlDf.loc[motherXmlDf['part'] == str(part)].index)
    for chap in ChapList:
        print(f"Chap {chap}")
        root[int(part)-1].append(etree.Element("chap", n=str(ch), name=str(chap)))
        ParaList = list(motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chap'] == chap)].index)
        for para in ParaList:
            print(f"Para {para}")
            root[part][chap].append(etree.Element("para", n=str(pa), name=str(para)))
            para_text = motherXmlDf.loc[pa].text
            print(f"Paratext: {para_text}")
            

prose
Part 1
Part 2


In [ ]:
motherXmlDf = motherTextDf['text'].str.split(' \n', expand=True).stack().to_frame().reset_index().rename(columns={'level_0':'chapID','level_1':'para',0:'text'})
motherXmlDf = motherXmlDf.loc[~motherXmlDf.text.str.contains(r"^\W*$", regex=True)]
motherXmlDf['text'] = motherXmlDf.text.str.replace('\n|\s{2,}', '')
motherXmlDf['part'] = motherXmlDf.chapID.apply(lambda x: int('1') if x < 30 else int('2'))
motherXmlDf['chap'] = motherXmlDf.chapID.map(motherTextDf['chapID'].to_dict())
motherXmlDf['para'] = motherXmlDf['para'].apply(lambda x: x+1)
motherXmlDf['paraID'] = range(1, len(motherXmlDf)+1)
motherXmlDf = motherXmlDf.reset_index().drop('index', axis=1)

motherXmlDf